# 1. Define a research question:

## Which community districts in New York City show the highest number of 311 complaints?

# 2. What data are available?

## 311 Service Requests from 2010 to the Present, https://datahub.cusp.nyu.edu/dataset/erm2-nwe9

In [1]:
# import packages
import os
import pandas as pd
import numpy as np
import re

In [2]:
# import dataset
data = pd.read_csv('/projects/open/NYCOpenData/nycopendata/data/erm2-nwe9/1446832678/erm2-nwe9')

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (8,17,39,40,47,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
complaints = data
print (complaints.columns)
complaints.head()

Index([u'Unique Key', u'Created Date', u'Closed Date', u'Agency',
       u'Agency Name', u'Complaint Type', u'Descriptor', u'Location Type',
       u'Incident Zip', u'Incident Address', u'Street Name', u'Cross Street 1',
       u'Cross Street 2', u'Intersection Street 1', u'Intersection Street 2',
       u'Address Type', u'City', u'Landmark', u'Facility Type', u'Status',
       u'Due Date', u'Resolution Description',
       u'Resolution Action Updated Date', u'Community Board', u'Borough',
       u'X Coordinate (State Plane)', u'Y Coordinate (State Plane)',
       u'Park Facility Name', u'Park Borough', u'School Name',
       u'School Number', u'School Region', u'School Code',
       u'School Phone Number', u'School Address', u'School City',
       u'School State', u'School Zip', u'School Not Found',
       u'School or Citywide Complaint', u'Vehicle Type',
       u'Taxi Company Borough', u'Taxi Pick Up Location',
       u'Bridge Highway Name', u'Bridge Highway Direction', u'Road Ramp',

,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,Incident Address,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,31911011,11/05/2015 02:59:15 AM,NaN,DOT,Department of Transportation,Street Condition,Pothole,NaN,11224,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.573431,-73.991742,"(40.57343122248129, -73.99174247588253)"
1,31908754,11/05/2015 02:09:49 AM,NaN,CHALL,CHALL,Opinion for the Mayor,HOUSING,NaN,NaN,NaN,...,NaN,NaN,NaN,1-1-1173130914,NaN,NaN,NaN,NaN,NaN,NaN
2,31910423,11/05/2015 02:06:51 AM,NaN,DPR,Department of Parks and Recreation,Root/Sewer/Sidewalk Condition,Trees and Sidewalks Program,Street,11234,1157 EAST 57 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.625004,-73.920726,"(40.62500363580505, -73.92072558378698)"
3,31909924,11/05/2015 02:02:20 AM,NaN,NYPD,New York City Police Department,Illegal Parking,Blocked Hydrant,Street/Sidewalk,11218,722 EAST 4 STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40.634522,-73.974790,"(40.634522428879706, -73.97479041437481)"
4,31913310,11/05/2015 01:57:20 AM,11/05/2015 01:57:31 AM,HRA,HRA Benefit Card Replacement,Benefit Card Replacement,Medicaid,NYC Street Address,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 3. Think about possible measurement errors

In [4]:
# What are the unique values for community districts?

CommDist = complaints['Community Board'].unique()

In [5]:
print ('There are %i unique values for community board in this dataset.' %len(CommDist))
print (CommDist)

There are 77 unique values for community board in this dataset.
['13 BROOKLYN' '0 Unspecified' '18 BROOKLYN' '12 BROOKLYN'
 '02 STATEN ISLAND' '08 MANHATTAN' '06 BROOKLYN' '10 BRONX' '06 MANHATTAN'
 '04 BROOKLYN' '04 QUEENS' '09 MANHATTAN' '14 BROOKLYN' '05 MANHATTAN'
 '02 MANHATTAN' '10 QUEENS' '07 QUEENS' '02 QUEENS' '11 BROOKLYN'
 '05 QUEENS' '05 BROOKLYN' '09 QUEENS' '12 MANHATTAN' '01 BROOKLYN'
 '11 MANHATTAN' '03 BRONX' '03 STATEN ISLAND' '08 BROOKLYN' '03 BROOKLYN'
 '07 MANHATTAN' '10 MANHATTAN' '04 BRONX' '01 MANHATTAN' '07 BRONX'
 '09 BRONX' '03 MANHATTAN' '13 QUEENS' '02 BRONX' '12 QUEENS' '16 BROOKLYN'
 '08 QUEENS' '04 MANHATTAN' '14 QUEENS' '10 BROOKLYN' '11 QUEENS'
 '06 BRONX' '08 BRONX' '05 BRONX' '15 BROOKLYN' '02 BROOKLYN' '12 BRONX'
 '03 QUEENS' '06 QUEENS' '01 STATEN ISLAND' '01 BRONX' '01 QUEENS'
 '17 BROOKLYN' '11 BRONX' 'Unspecified QUEENS' '09 BROOKLYN'
 'Unspecified MANHATTAN' 'Unspecified BROOKLYN' '07 BROOKLYN'
 'Unspecified BRONX' '64 MANHATTAN' '81 QUEENS' '5

#### There are 59 community districts in NYC, but this dataset has 77 unique values for community district

In [6]:
CommDist.sort()
print (CommDist)

['0 Unspecified' '01 BRONX' '01 BROOKLYN' '01 MANHATTAN' '01 QUEENS'
 '01 STATEN ISLAND' '02 BRONX' '02 BROOKLYN' '02 MANHATTAN' '02 QUEENS'
 '02 STATEN ISLAND' '03 BRONX' '03 BROOKLYN' '03 MANHATTAN' '03 QUEENS'
 '03 STATEN ISLAND' '04 BRONX' '04 BROOKLYN' '04 MANHATTAN' '04 QUEENS'
 '05 BRONX' '05 BROOKLYN' '05 MANHATTAN' '05 QUEENS' '06 BRONX'
 '06 BROOKLYN' '06 MANHATTAN' '06 QUEENS' '07 BRONX' '07 BROOKLYN'
 '07 MANHATTAN' '07 QUEENS' '08 BRONX' '08 BROOKLYN' '08 MANHATTAN'
 '08 QUEENS' '09 BRONX' '09 BROOKLYN' '09 MANHATTAN' '09 QUEENS' '10 BRONX'
 '10 BROOKLYN' '10 MANHATTAN' '10 QUEENS' '11 BRONX' '11 BROOKLYN'
 '11 MANHATTAN' '11 QUEENS' '12 BRONX' '12 BROOKLYN' '12 MANHATTAN'
 '12 QUEENS' '13 BROOKLYN' '13 QUEENS' '14 BROOKLYN' '14 QUEENS'
 '15 BROOKLYN' '16 BROOKLYN' '17 BROOKLYN' '18 BROOKLYN' '26 BRONX'
 '27 BRONX' '28 BRONX' '55 BROOKLYN' '56 BROOKLYN' '64 MANHATTAN'
 '80 QUEENS' '81 QUEENS' '82 QUEENS' '83 QUEENS' '84 QUEENS'
 '95 STATEN ISLAND' 'Unspecified BRONX' 'Unsp

#### The real 3 districts for Staten Island, 12 each for Bronx and Manhattan, 14 for Queens, and 18 for Brooklyn (https://www1.nyc.gov/site/planning/community/community-portal.page) are all included in the above dataset. The additional community districts numbered above 18, and "0 Unspecified", are all inaccurate.

## Are there duplicates in this dataset? Are these plausible?

In [7]:
print ("There are %i complaints in this dataset." %len(complaints))
print ("There are %i unique keys for complaints in this dataset." %len(complaints['Unique Key'].unique()))
if len(complaints) > len(complaints['Unique Key'].unique()):
    print ("There are fewer 'Unique Key' values than total rows in the dataset, meaning you have some duplicate complaints.")
else: print ("There are an equal number of 'Unique Key' values as total rows in the dataset, meaning there are no duplicates.")

There are 10187766 complaints in this dataset.
There are 10187744 unique keys for complaints in this dataset.
There are fewer 'Unique Key' values than total rows in the dataset, meaning you have some duplicate complaints.


#### Duplicate complaints can be plausible in situations where multiple people call about the same issue. These should be dropped.

In [8]:
# Check that code drops only duplicate rows rather than both
# UniqComplaints = complaints.drop_duplicates(subset='Unique Key', keep='first')
# print len(UniqComplaints['Unique Key'])

# Drop duplicates from actual dataframe
complaints = complaints.drop_duplicates(subset='Unique Key', keep='first')
print ("There are %i complaints in this dataset." %len(complaints))

There are 10187744 complaints in this dataset.


## Are there missing values? Are there patterns in the missing values?

In [11]:
# Check how many rows have a null value for each column
checkNull = complaints.isnull().sum() 

In [40]:
checkNull.sort()
print (checkNull)

School or Citywide Complaint      10185390
Ferry Direction                   10185001
Garage Lot Name                   10183411
Ferry Terminal Name               10181220
Landmark                          10180434
Vehicle Type                      10180242
Taxi Company Borough              10177579
Road Ramp                         10152964
Bridge Highway Direction          10152701
Bridge Highway Name               10152649
Bridge Highway Segment            10135758
Taxi Pick Up Location             10089185
Facility Type                      8579463
Intersection Street 2              8346281
Intersection Street 1              8345267
School Not Found                   7151698
Due Date                           7120286
Resolution Description             3786739
Location Type                      3067596
Cross Street 2                     2425792
Cross Street 1                     2375942
Street Name                        2198558
Incident Address                   2197705
Y Coordinat

#### A quick glance at the above print out shows that 12 columns are almost entirely null. However, there are also 14 columns without a single null value, 2 school-related columns with fewer than 10 null values, and the remaining columns vary greatly. It is likely that the columns without any null values are either automatically generated or cannot be left null by the person entering call information into the dataset. Those that are almost entirely null may be optional information that callers are not prompted to provide and typically don't think to provide themselves. Another pattern that jumps out is that the 4 coordinate columns and "Location" all have the same number of missing values, suggesting that these columns are related to each other and automatically generated.

## Data Editing: Can missing values be replaced? Can missing "Resolution Descriptor" values be replaced with "Complaint Type" data?

In [54]:
# Compare the values of some rows for these two columns

print (complaints['Complaint Type'][523:530], complaints['Resolution Description'][523:530])
print (complaints['Resolution Description'][527], complaints['Resolution Description'][526])

(523        HEAT/HOT WATER
524      Blocked Driveway
525    Indoor Air Quality
526    Noise - Commercial
527       Noise - Vehicle
528      Blocked Driveway
529                 Noise
Name: Complaint Type, dtype: object, 523    The complaint you filed is a duplicate of a co...
524    Your complaint has been received by the Police...
525    The Department of Health and Mental Hygiene wi...
526    The Police Department responded to the complai...
527    The Police Department responded to the complai...
528    The Police Department issued a summons in resp...
529                                                  NaN
Name: Resolution Description, dtype: object)
('The Police Department responded to the complaint and with the information available observed no evidence of the violation at that time.', 'The Police Department responded to the complaint and with the information available observed no evidence of the violation at that time.')


#### The values in the "Resolution Description" column can certainly be set to equal the values of the "Complaint Type" column, however I do not believe this would be partiularly useful for analytical purposes. It would be more beneficial to determine if some complaint types are more or less likely to have a resolution description. You could also check to see if specific complaint types with a specific status have one specific resolution description. If that is the case, it would be possible and potentially beneficial to set the rows with empty resolution descriptions that match a given complaint type-status combination to that combination's given resolution description.

## Create a marker for implausible Community Districts and then drop them.

In [ ]:
# This was code to rename the community districts so as to have all inaccurate districts in a single group.
# Lab instructions were to instead create a marker for the inaccurate districts and drop them.
'''
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('Unspecified MANHATTAN', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('Unspecified BRONX', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('Unspecified BROOKLYN', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('Unspecified QUEENS', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: 
                                                                  x.replace('Unspecified STATEN ISLAND', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('0 Unspecified', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('26 BRONX', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('27 BRONX', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('28 BRONX', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('55 BROOKLYN', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('56 BROOKLYN', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('64 MANHATTAN', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('80 QUEENS', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('81 QUEENS', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('82 QUEENS', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('83 QUEENS', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('84 QUEENS', 'Unspecified'))
complaints['Community Board'] = complaints['Community Board'].map(lambda x: x.replace('95 STATEN ISLAND', 'Unspecified'))

CommDist = complaints['Community Board'].unique()
CommDist.sort()
print ('''There are now %i unique values for community board in this dataset, one for each actual community board
       and one for the rest.''' %len(CommDist))
print (CommDist)
'''

In [76]:
# Group the complaints by Community Board regardless of whether it is valid.
dist311 = complaints.groupby('Community Board', as_index=False).count()

In [77]:
dist311.head()

,Community Board,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,...,Bridge Highway Name,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location
0,0 Unspecified,1109446,1109446,1049131,1109446,1109446,1109446,1109160,882275,827262,...,285,247,278,17539,0,2743,4611,820850,820850,820850
1,01 BRONX,74631,74631,71419,74631,74631,74631,74312,59752,74435,...,410,410,410,410,1,0,39,73748,73748,73748
2,01 BROOKLYN,185057,185057,178247,185057,185057,185057,184119,127919,184497,...,994,994,994,994,2,0,41,182790,182790,182790
3,01 MANHATTAN,77974,77974,74510,77974,77974,77974,77099,42828,77877,...,350,350,267,267,65,0,18,77186,77186,77186
4,01 QUEENS,171484,171484,164598,171484,171484,171484,170936,121938,170826,...,692,692,692,692,457,0,35,169154,169154,169154


In [80]:
# Create a marker
dist311['Valid'] = np.ones(len(dist311), np.float)
for i in range (0, len(dist311)):
    try:
        if int(dist311['Community Board'][i].split(' ')[0]) > 18:
            dist311['Valid'][i] = "False"
        elif int(dist311['Community Board'][i].split(' ')[0]) == 0:
            dist311['Valid'][i] = "False"
        elif int(dist311['Community Board'][i].split(' ')[0]) > 0 and int(dist311['Community Board'][i].split(' ')[0]) < 19:
            dist311['Valid'][i] = "True"
    except ValueError:
        dist311['Valid'][i] = "False"

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/pandas/core/indexing.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/rh/anaconda/root

In [81]:
# Drop invalid districts
dist311 = dist311[(dist311['Valid'] == "True")]
print (len(dist311))
dist311.head()

59


,Community Board,Unique Key,Created Date,Closed Date,Agency,Agency Name,Complaint Type,Descriptor,Location Type,Incident Zip,...,Bridge Highway Direction,Road Ramp,Bridge Highway Segment,Garage Lot Name,Ferry Direction,Ferry Terminal Name,Latitude,Longitude,Location,Valid
1,01 BRONX,74631,74631,71419,74631,74631,74631,74312,59752,74435,...,410,410,410,1,0,39,73748,73748,73748,True
2,01 BROOKLYN,185057,185057,178247,185057,185057,185057,184119,127919,184497,...,994,994,994,2,0,41,182790,182790,182790,True
3,01 MANHATTAN,77974,77974,74510,77974,77974,77974,77099,42828,77877,...,350,267,267,65,0,18,77186,77186,77186,True
4,01 QUEENS,171484,171484,164598,171484,171484,171484,170936,121938,170826,...,692,692,692,457,0,35,169154,169154,169154,True
5,01 STATEN ISLAND,182708,182708,171750,182708,182708,182708,182173,121503,182038,...,538,538,538,396,0,42,180795,180795,180795,True


## Rank the discticts according to the number of complaints

In [92]:
# Reduce the dataframe and rank the districts
ComplaintsPerCD = dist311[['Community Board', 'Unique Key']]
ComplaintsPerCD.rename(columns = {'Unique Key': 'Complaints'}, inplace = True)
ComplaintsPerCD['Rank'] = ComplaintsPerCD['Complaints'].rank(ascending = False)
ComplaintsPerCD.sort(columns = 'Rank', inplace=True)
ComplaintsPerCD.head()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/pandas/core/frame.py:3225: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  na_position=na_position)


,Community Board,Complaints,Rank
50,12 MANHATTAN,268053,1.0
51,12 QUEENS,229383,2.0
12,03 BROOKLYN,197305,3.0
31,07 QUEENS,195961,4.0
58,17 BROOKLYN,191720,5.0


# 4. Think about the interpretation of your results.

In [93]:
# Load the Population by community district dataset
pop = pd.read_csv('http://cosmo.nyu.edu/~fb55/PUI2016/data/Final_Demographics.csv')

In [94]:
pop.head()

,FIPS,cd_id,Total Population,Population Density (per sq. mile),% Total Population: Male,% Total Population: 18 to 24 Years,% Total Population: 25 to 34 Years,% Total Population: 35 to 44 Years,% Population 5 Years And Over: Speak Only English,% Population 5 Years And Over: Spanish or Spanish Creole,...,"Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations","% Employed Civilian Population 16 Years And Over: Management, professional, and related occupations",% Employed Civilian Population 16 Years And Over: Service occupations,% Employed Civilian Population 16 Years And Over: Sales and office occupations,"% Employed Civilian Population 16 Years And Over: Farming, fishing, and forestry occupations","% Employed Civilian Population 16 Years And Over: Construction, extraction, and maintenance occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Production occupations","% Employed Civilian Population 16 Years And Over: Production, transportation, and material moving occupations: Transportation and material moving occupations"
0,3603701,BX08,106737,31229.95006,46.65,10.73,15.04,11.32,46.80,39.24,...,665,1518,42.66,28.95,20.89,0.24,2.65,4.60,1.40,3.20
1,3603702,BX12,134644,19966.67839,46.35,11.35,14.29,12.57,73.09,18.19,...,1156,4174,29.57,33.98,20.40,0.00,7.08,8.97,1.95,7.02
2,3603703,BX10,121209,12913.81703,45.20,8.62,13.74,12.78,61.79,26.43,...,941,3433,36.20,22.85,25.09,0.00,7.68,8.18,1.76,6.42
3,3603704,BX11,135839,35677.95453,50.09,8.10,17.43,14.09,43.22,36.45,...,2189,5592,30.06,27.86,22.24,0.00,7.03,12.81,3.60,9.20
4,3603705,BX03,172247,39405.79222,44.72,14.24,14.89,12.38,36.82,54.24,...,1437,5436,16.80,41.00,22.29,0.03,8.45,11.43,2.39,9.04


In [95]:
cd_id = pop['cd_id'].unique()
print ("There are %i unique community districts in this dataset" %len(pop['cd_id'].unique()))
print (cd_id)

There are 59 unique community districts in this dataset
['BX08' 'BX12' 'BX10' 'BX11' 'BX03' 'BX06' 'BX07' 'BX05' 'BX04' 'BX09'
 'BX01' 'BX02' 'MN12' 'MN09' 'MN10' 'MN11111' 'MN08' 'MN07' 'MN05' 'MN04'
 'MN06' 'MN03' 'MN01' 'MN02' 'SI03' 'SI02' 'SI01' 'BK01' 'BK04' 'BK03'
 'BK02' 'BK06' 'BK08' 'BK16' 'BK05' 'BK18' 'BK17' 'BK09' 'BK07' 'BK10'
 'BK12' 'BK14' 'BK15' 'BK11' 'BK13' 'QN01' 'QN03' 'QN07' 'QN11' 'QN13'
 'QN08' 'QN04' 'QN06' 'QN02' 'QN05' 'QN09' 'QN12' 'QN10' 'QN14']


## This dataset has 59 unique community districts, which is the correct amount. But we still need to check to make sure that the districts themselves are correct.

In [ ]:
# Before fully reading the insructions, was going to format districts in all datasets as "## Borough"  
'''
pop['cd_id'] = pop['cd_id'].map(lambda x: x.replace('BX', 'BRONX '))
pop['cd_id'] = pop['cd_id'].map(lambda x: x.replace('MN', 'MANHATTAN '))
pop['cd_id'] = pop['cd_id'].map(lambda x: x.replace('SI', 'STATEN_ISLAND '))
pop['cd_id'] = pop['cd_id'].map(lambda x: x.replace('BK', 'BROOKLYN '))
pop['cd_id'] = pop['cd_id'].map(lambda x: x.replace('QN', 'QUEENS '))
cd_id = pop['cd_id'].unique()
print (pop['cd_id'].unique())
'''

In [ ]:
# Continuation of above process
'''
pop['commdist'] = None
for i in range (0, len(pop)):
    pop['commdist'][i] = pop['cd_id'][i].split(' ')[1] + ' ' + pop['cd_id'][i].split(' ')[0]
print (pop['commdist'].unique())
'''

In [96]:
cd_id.sort()
print (cd_id)

['BK01' 'BK02' 'BK03' 'BK04' 'BK05' 'BK06' 'BK07' 'BK08' 'BK09' 'BK10'
 'BK11' 'BK12' 'BK13' 'BK14' 'BK15' 'BK16' 'BK17' 'BK18' 'BX01' 'BX02'
 'BX03' 'BX04' 'BX05' 'BX06' 'BX07' 'BX08' 'BX09' 'BX10' 'BX11' 'BX12'
 'MN01' 'MN02' 'MN03' 'MN04' 'MN05' 'MN06' 'MN07' 'MN08' 'MN09' 'MN10'
 'MN11111' 'MN12' 'QN01' 'QN02' 'QN03' 'QN04' 'QN05' 'QN06' 'QN07' 'QN08'
 'QN09' 'QN10' 'QN11' 'QN12' 'QN13' 'QN14' 'SI01' 'SI02' 'SI03']


### Other than one Manhattan Community District, all are correct. 

In [97]:
pop['cd_id'] = pop['cd_id'].map(lambda x: x.replace('MANHATTAN 11111', 'MANHATTAN 11'))
cd_id = pop['cd_id'].unique()
cd_id.sort()
print (cd_id)

['BK01' 'BK02' 'BK03' 'BK04' 'BK05' 'BK06' 'BK07' 'BK08' 'BK09' 'BK10'
 'BK11' 'BK12' 'BK13' 'BK14' 'BK15' 'BK16' 'BK17' 'BK18' 'BX01' 'BX02'
 'BX03' 'BX04' 'BX05' 'BX06' 'BX07' 'BX08' 'BX09' 'BX10' 'BX11' 'BX12'
 'MN01' 'MN02' 'MN03' 'MN04' 'MN05' 'MN06' 'MN07' 'MN08' 'MN09' 'MN10'
 'MN11111' 'MN12' 'QN01' 'QN02' 'QN03' 'QN04' 'QN05' 'QN06' 'QN07' 'QN08'
 'QN09' 'QN10' 'QN11' 'QN12' 'QN13' 'QN14' 'SI01' 'SI02' 'SI03']


### Who might be more likely to call 911?

In [ ]:
# Load the infrastructure by community district dataset
infra_struct = pd.read_csv("http://cosmo.nyu.edu/~fb55/PUI2016/data/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv")

In [ ]:
print (len(infra_struct))
print (infra_struct.columns)
#infra_struct.head()

In [ ]:
uniqDist = infra_struct['Qualifying Name'].unique()
print (len(uniqDist))
print (uniqDist)

In [ ]:
infra_struct['BoardNum'] = np.ones(len(infra_struct), np.float)
for i in range (0, len(infra_struct)):
    infra_struct['BoardNum'][i] = str(infra_struct['Qualifying Name'][i]).split('District ')[1].split('--')[0] + " " + str(infra_struct['Qualifying Name'][i]).split('-')[1].split(' ')[0]
print (infra_struct['BoardNum'])

In [ ]:
infra_struct['valid'] = np.ones(len(infra_struct), np.float)
for i in range (0, len(infra_struct)):
    try:
        if int(infra_struct['BoardNum'][i].split(' &')[0]) < 10:
            infra_struct['valid'][i] = 'False'
    except ValueError:
        if int(infra_struct['BoardNum'][i].split(' ')[0]) < 10:
            infra_struct['BoardNum'][i] = "0" + infra_struct['BoardNum'][i]
            infra_struct['valid'][i] = "True"
        else: infra_struct['valid'][i] = "True"

In [ ]:
#test = infra_struct[['BoardNum', 'valid']]
#print (test)

In [ ]:
for i in range (0, len(infra_struct)):
    try:
        if infra_struct['valid'][i] == 'False':
            infra_struct['BoardNum'] = 
        if int(infra_struct['BoardNum'][i].split(' ')[0]) < 10:
            infra_struct['BoardNum'][i] = "0" + infra_struct['BoardNum'][i]
            infra_struct['valid'][i] = "True"
        else: infra_struct['valid'][i] = "True"